In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats
import seaborn as sns

%matplotlib notebook

In [2]:
# Define the functions for water footprint calculation
def get_gwf_df(base_path, irrg_style, area_rate, rice_yield):
    gwf_list = []
    for run_id in range(0,100):
        loads_file = "{}{}loads_4gwf{}.txt".format(base_path, irrg_style, run_id)
        loads_df = pd.read_csv(loads_file, skiprows=1, sep="\t", header=0, index_col=0)
        gwf_df = pd.DataFrame({"runoff_N": loads_df["TN_runoff"] / 0.002, "runoff_P": loads_df["TP_runoff"] / 0.0004,
                               "leaching": loads_df["TN_leaching"] / 0.02})   # in m3/ha
        gwf_dff = pd.DataFrame({"runoff_Nf": loads_df["TN_fs"] / 0.002, "runoff_Pf": loads_df["TP_fs"] / 0.0004})
        gwf_df["runoff"] = gwf_df[["runoff_N", "runoff_P"]].max(axis=1)
        gwf_dff["runoff_f"] = gwf_dff[["runoff_Nf", "runoff_Pf"]].max(axis=1)
        gwf_df1 = gwf_df / (rice_yield * (1 - area_rate))  # m3/ton
        gwf_df1["runoff_f"] = gwf_dff["runoff_f"] / rice_yield  # m3/ton
        gwf_list.append(gwf_df1)
    gwf_df_new = pd.concat(gwf_list, ignore_index=True)
    return gwf_df_new


def get_gbwf_df(base_path, irrg_style, area_rate, rice_yield, start_day, end_day):
    I_out_file = "{}{}I_out50.txt".format(base_path, irrg_style)
    I_out = pd.read_csv(I_out_file, skiprows=1, sep="\t", header=0, index_col=0)
    prcp_file = "{}{}prcp_A.txt".format(base_path, irrg_style)
    prcp = pd.read_csv(prcp_file, skiprows=1, sep="\t", header=0, index_col=0)
    et_file = "{}{}ET_d.txt".format(base_path, irrg_style)
    et_d = pd.read_csv(et_file, sep="\t", skiprows=1, header=0, index_col=0, parse_dates=True)
    pet = et_d.resample("A", kind="period").sum()
    if irrg_style in ["totally centralized/", "partly centralized 2/"]:
        bwf = pet * (I_out / (I_out + prcp)).values
        gwf = pet * (prcp / (I_out + prcp)).values
        gbwf_df = pd.DataFrame({"gwf": gwf.values.reshape(-1) * 10, "bwf":bwf.values.reshape(-1) * 10})  # in m3/ha
    else:
        I_all_file = "{}{}I_all50.txt".format(base_path, irrg_style)
        I_all = pd.read_csv(I_all_file, skiprows=1, sep="\t", header=0, index_col=0)
        bwf = pet * (I_out / (I_all + prcp)).values
        gwf = pet * (prcp / (I_all + prcp)).values
        greywf = bwf + gwf - pet
        gbwf_df = pd.DataFrame({"gwf": gwf.values.reshape(-1) * 10, "bwf": bwf.values.reshape(-1) * 10, 
                                "greywf": greywf.values.reshape(-1) * 10})  # in m3/ha
    gbwf_df1 = gbwf_df / rice_yield  # in m3/ton
    return gbwf_df1

In [3]:
def get_wf_df(gwf_df, gbwf_df, irrg_style):
    wf_df = gwf_df.copy(deep=True)
    gbwf_df1 = pd.concat([gbwf_df] * 100)
    wf_df["gwf"] = gbwf_df1["gwf"].values
    wf_df["bwf"] = gbwf_df1["bwf"].values
    if irrg_style in ["totally decentralized/", "partly centralized 1/"]:
        wf_df["greywf"] = gbwf_df1["greywf"].values
    wf_df["total"] = wf_df["gwf"] + wf_df["bwf"] + wf_df["runoff"] + wf_df["leaching"]
    wf_df["graywf"] = wf_df["runoff"] + wf_df["leaching"]
    return wf_df

def filter_wf_df(wf_df, wf_df_check):
    # Check and remove extreme values for runoff gwf(greater than 95%)
    stats_df = wf_df_check.quantile(q=0.95)
    wf_df["runoff_filtered"] = wf_df["runoff"].where(wf_df_check["runoff"] < (stats_df.loc["runoff"]), np.NAN)
    wf_df["runoff_f_filtered"] = wf_df["runoff_f"].where(wf_df_check["runoff_f"] < (stats_df.loc["runoff_f"]), np.NAN)
    wf_df["total_filtered"] = wf_df["gwf"] + wf_df["bwf"] + wf_df["runoff_filtered"] + wf_df["leaching"]
    return wf_df


In [4]:
def wf_province(province_name, area_rate_dic, rice_yield, start_day, end_day):
    base_path = "C:/Users/Dell/PycharmProjects/WQQM_PIDU/wf_new/{}/".format(province_name)
    irrg_style_dic = {"TC": "totally centralized/", "TD": "totally decentralized/"}
    wf_mean_dic = {}
    wf_std_dic = {}
    wf_p5_dic = {}
    wf_p95_dic = {}
    
    for key, irrg_style in irrg_style_dic.items():
        gwf_df = get_gwf_df(base_path, irrg_style, area_rate_dic[irrg_style], rice_yield)
        gbwf_df = get_gbwf_df(base_path, irrg_style, area_rate_dic[irrg_style], rice_yield, start_day, end_day)
        wf_df = get_wf_df(gwf_df, gbwf_df, irrg_style)
        gbwf_df.to_csv("{}{}gbwf.txt".format(base_path, irrg_style), sep="\t", header=True, index=True, float_format="%.2f")
        wf_df.to_csv("{}{}wf.txt".format(base_path, irrg_style), sep="\t", header=True, index=True, float_format="%.2f")
        wf_mean_dic[key] = wf_df.mean()
        wf_std_dic[key] = wf_df.std()
        wf_p5_dic[key] = wf_df.quantile(q=0.05)
        wf_p95_dic[key] = wf_df.quantile(q=0.95)
    
    wf_mean_df = pd.DataFrame(wf_mean_dic)
    wf_std_df = pd.DataFrame(wf_std_dic)
    wf_p5_df = pd.DataFrame(wf_p5_dic)
    wf_p95_df = pd.DataFrame(wf_p95_dic)
    return wf_mean_df, wf_std_df, wf_p5_df, wf_p95_df

In [5]:
rice_yield_dic = {"Heilongjiang": 7.085, "Jilin": 8.100, "Liaoning": 7.990, "Jiangsu": 8.402, "Anhui": 6.539, "Hubei": 8.423, "Sichuan": 7.717, "Guizhou": 5.651, "Yunnan": 6.049,
                 "Jiangxi2S": 11.671, "Hunan2S": 12.220, "Hunan1S": 7.032, "Jiangxi1S": 6.500, "Zhejiang": 7.576, "Fujian1S": 6.322, "Guangdong": 11.331, "Guangxi": 11.095, "Hainan": 9.789, "Fujian2S": 12.046}
area_rate_dic = {"Heilongjiang": {"totally centralized/": 0.03602, "totally decentralized/": 0.03602, "partly centralized 1/": 0.03602, "partly centralized 2/": 0.03602},
                  "Jilin": {"totally centralized/": 0.02920, "totally decentralized/": 0.03249, "partly centralized 1/": 0.02920, "partly centralized 2/": 0.03249}, 
                  "Liaoning": {"totally centralized/": 0.04064, "totally decentralized/": 0.05291, "partly centralized 1/": 0.04064, "partly centralized 2/": 0.05291}, 
                  "Jiangsu": {"totally centralized/": 0.07157, "totally decentralized/": 0.07654, "partly centralized 1/": 0.07157, "partly centralized 2/": 0.07654}, 
                  "Anhui": {"totally centralized/": 0.05660, "totally decentralized/": 0.07382, "partly centralized 1/": 0.05660, "partly centralized 2/": 0.07382}, 
                  "Hubei": {"totally centralized/": 0.05101, "totally decentralized/": 0.07983, "partly centralized 1/": 0.05101, "partly centralized 2/": 0.07983}, 
                  "Sichuan": {"totally centralized/": 0.01599, "totally decentralized/": 0.01744, "partly centralized 1/": 0.01599, "partly centralized 2/": 0.01744}, 
                  "Guizhou": {"totally centralized/": 0.02029, "totally decentralized/": 0.02029, "partly centralized 1/": 0.02029, "partly centralized 2/": 0.02029}, 
                  "Yunnan": {"totally centralized/": 0.02121, "totally decentralized/": 0.04110, "partly centralized 1/": 0.02121, "partly centralized 2/": 0.04110},
                  "Jiangxi2S": {"totally centralized/": 0.05213, "totally decentralized/": 0.08004, "partly centralized 1/": 0.05213, "partly centralized 2/": 0.08004}, 
                  "Hunan2S": {"totally centralized/": 0.04038, "totally decentralized/": 0.08053, "partly centralized 1/": 0.04038, "partly centralized 2/": 0.08053},
                  "Jiangxi1S": {"totally centralized/": 0.05213, "totally decentralized/": 0.08004, "partly centralized 1/": 0.05213, "partly centralized 2/": 0.08004}, 
                  "Hunan1S": {"totally centralized/": 0.04038, "totally decentralized/": 0.08053, "partly centralized 1/": 0.04038, "partly centralized 2/": 0.08053},
                  "Zhejiang": {"totally centralized/": 0.03846, "totally decentralized/": 0.05971, "partly centralized 1/": 0.03846, "partly centralized 2/": 0.05971}, 
                  "Fujian1S": {"totally centralized/": 0.05417, "totally decentralized/": 0.05924, "partly centralized 1/": 0.05417, "partly centralized 2/": 0.05924}, 
                  "Guangdong": {"totally centralized/": 0.03607, "totally decentralized/": 0.08022, "partly centralized 1/": 0.03607, "partly centralized 2/": 0.08022}, 
                  "Guangxi": {"totally centralized/": 0.03069, "totally decentralized/": 0.03397, "partly centralized 1/": 0.03069, "partly centralized 2/": 0.03397}, 
                  "Hainan": {"totally centralized/": 0.07386, "totally decentralized/": 0.07974, "partly centralized 1/": 0.07386, "partly centralized 2/": 0.07974}, 
                  "Fujian2S": {"totally centralized/": 0.05417, "totally decentralized/": 0.05924, "partly centralized 1/": 0.05417, "partly centralized 2/": 0.05924}}
start_day_dic = {"Heilongjiang": "5-5", "Jilin": "5-5", "Liaoning": "5-10", "Jiangsu": "5-27", "Anhui": "5-27", "Hubei": "5-27", "Sichuan": "5-27", "Guizhou": "5-27", "Yunnan": "5-27",
                 "Jiangxi2S": "4-16", "Hunan2S": "4-16", "Hunan1S": "5-27", "Jiangxi1S": "5-27", "Zhejiang": "6-15", "Fujian1S": "6-15", "Guangdong": "4-10", "Guangxi": "4-10", "Hainan": "4-10", "Fujian2S": "4-10"}
end_day_dic = {"Heilongjiang": "9-25", "Jilin": "9-25", "Liaoning": "10-5", "Jiangsu": "9-20", "Anhui": "9-20", "Hubei": "9-20", "Sichuan": "9-20", "Guizhou": "9-20", "Yunnan": "9-20",
                 "Jiangxi2S": "11-10", "Hunan2S": "11-10", "Hunan1S": "9-20", "Jiangxi1S": "9-20", "Zhejiang": "11-20", "Fujian1S": "11-20", "Guangdong": "11-10", "Guangxi": "11-10", "Hainan": "11-10", "Fujian2S": "11-10"}

# Calculate WF for rice-production provinces

In [8]:
wf_mean_TC_dic = {}
wf_mean_TD_dic = {}
wf_std_TC_dic = {}
wf_std_TD_dic = {}
wf_p5_TC_dic = {}
wf_p5_TD_dic = {}
wf_p95_TC_dic = {}
wf_p95_TD_dic = {}
for province, rice_yield in rice_yield_dic.items():
    area_rate_dic1 = area_rate_dic[province]
    start_day = start_day_dic[province]
    end_day = end_day_dic[province]
    wf_mean, wf_std, wf_p5, wf_p95 = wf_province(province, area_rate_dic1, rice_yield, start_day, end_day)
    wf_mean_TC_dic[province] = wf_mean["TC"]
    wf_mean_TD_dic[province] = wf_mean["TD"]
    wf_std_TC_dic[province] = wf_std["TC"]
    wf_std_TD_dic[province] = wf_std["TD"]
    wf_p5_TC_dic[province] = wf_p5["TC"]
    wf_p5_TD_dic[province] = wf_p5["TD"]
    wf_p95_TC_dic[province] = wf_p95["TC"]
    wf_p95_TD_dic[province] = wf_p95["TD"]

wf_mean_TC = pd.DataFrame(wf_mean_TC_dic)
wf_mean_TD = pd.DataFrame(wf_mean_TD_dic)
wf_std_TC = pd.DataFrame(wf_std_TC_dic)
wf_std_TD = pd.DataFrame(wf_std_TD_dic)
wf_p5_TC = pd.DataFrame(wf_p5_TC_dic)
wf_p5_TD = pd.DataFrame(wf_p5_TD_dic)
wf_p95_TC = pd.DataFrame(wf_p95_TC_dic)
wf_p95_TD = pd.DataFrame(wf_p95_TD_dic)

wf_mean_TC.to_csv("wf_results_2020.10/wf_mean_TC.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_mean_TD.to_csv("wf_results_2020.10/wf_mean_TD.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_std_TC.to_csv("wf_results_2020.10/wf_std_TC.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_std_TD.to_csv("wf_results_2020.10/wf_std_TD.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_p5_TC.to_csv("wf_results_2020.10/wf_p5_TC.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_p5_TD.to_csv("wf_results_2020.10/wf_p5_TD.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_p95_TC.to_csv("wf_results_2020.10/wf_95_TC.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_p95_TD.to_csv("wf_results_2020.10/wf_95_TD.txt", sep="\t", header=True, index=True, float_format="%.2f")

In [9]:
weight_yield_dic = {"Heilongjiang": 2571.3, "Jilin": 606.8, "Liaoning": 433.0, "Jiangsu": 1876.2, "Anhui": 1258.9, "Hubei": 1435.3, "Sichuan": 1959.4, "Guizhou": 398.5, "Yunnan": 518.9,
                 "Jiangxi2S": 1615.5, "Hunan2S": 1773.4, "Hunan1S": 888.5, "Jiangxi1S":441.6, "Zhejiang": 376.9, "Fujian1S": 169.7, "Guangdong": 1047.0, "Guangxi": 974.2, "Hainan": 134.9, "Fujian2S": 251.9}
sum_yield = 0
for weight_yield in weight_yield_dic.values():
    sum_yield += weight_yield
print(sum_yield, sum_yield/20601.1)

18731.900000000005 0.9092669808893703


# Calculate rice yield weighted WF for China

In [10]:
weight_yield_df = pd.DataFrame(weight_yield_dic, index=wf_mean_TD.index)
weight_yield_df = weight_yield_df / sum_yield
weight_yield_df

,Heilongjiang,Jilin,Liaoning,Jiangsu,Anhui,Hubei,Sichuan,Guizhou,Yunnan,Jiangxi2S,Hunan2S,Hunan1S,Jiangxi1S,Zhejiang,Fujian1S,Guangdong,Guangxi,Hainan,Fujian2S
bwf,0.137269,0.032394,0.023116,0.100161,0.067206,0.076623,0.104602,0.021274,0.027701,0.086243,0.094673,0.047432,0.023575,0.020121,0.009059,0.055894,0.052008,0.007202,0.013448
graywf,0.137269,0.032394,0.023116,0.100161,0.067206,0.076623,0.104602,0.021274,0.027701,0.086243,0.094673,0.047432,0.023575,0.020121,0.009059,0.055894,0.052008,0.007202,0.013448
greywf,0.137269,0.032394,0.023116,0.100161,0.067206,0.076623,0.104602,0.021274,0.027701,0.086243,0.094673,0.047432,0.023575,0.020121,0.009059,0.055894,0.052008,0.007202,0.013448
gwf,0.137269,0.032394,0.023116,0.100161,0.067206,0.076623,0.104602,0.021274,0.027701,0.086243,0.094673,0.047432,0.023575,0.020121,0.009059,0.055894,0.052008,0.007202,0.013448
leaching,0.137269,0.032394,0.023116,0.100161,0.067206,0.076623,0.104602,0.021274,0.027701,0.086243,0.094673,0.047432,0.023575,0.020121,0.009059,0.055894,0.052008,0.007202,0.013448
runoff,0.137269,0.032394,0.023116,0.100161,0.067206,0.076623,0.104602,0.021274,0.027701,0.086243,0.094673,0.047432,0.023575,0.020121,0.009059,0.055894,0.052008,0.007202,0.013448
runoff_N,0.137269,0.032394,0.023116,0.100161,0.067206,0.076623,0.104602,0.021274,0.027701,0.086243,0.094673,0.047432,0.023575,0.020121,0.009059,0.055894,0.052008,0.007202,0.013448
runoff_P,0.137269,0.032394,0.023116,0.100161,0.067206,0.076623,0.104602,0.021274,0.027701,0.086243,0.094673,0.047432,0.023575,0.020121,0.009059,0.055894,0.052008,0.007202,0.013448
runoff_f,0.137269,0.032394,0.023116,0.100161,0.067206,0.076623,0.104602,0.021274,0.027701,0.086243,0.094673,0.047432,0.023575,0.020121,0.009059,0.055894,0.052008,0.007202,0.013448
total,0.137269,0.032394,0.023116,0.100161,0.067206,0.076623,0.104602,0.021274,0.027701,0.086243,0.094673,0.047432,0.023575,0.020121,0.009059,0.055894,0.052008,0.007202,0.013448


In [11]:
wf_TC_df = wf_mean_TC * weight_yield_df
wf_TC_China = wf_TC_df.sum(axis=1)
wf_TD_df = wf_mean_TD * weight_yield_df
wf_TD_China = wf_TD_df.sum(axis=1)
wf_TC_p5_df = wf_p5_TC * weight_yield_df
wf_TD_p5_df = wf_p5_TD * weight_yield_df
wf_TC_p95_df = wf_p95_TC * weight_yield_df
wf_TD_p95_df = wf_p95_TD * weight_yield_df
wf_TC_p5_China = wf_TC_p5_df.sum(axis=1)
wf_TD_p5_China = wf_TD_p5_df.sum(axis=1)
wf_TC_p95_China = wf_TC_p95_df.sum(axis=1)
wf_TD_p95_China = wf_TD_p95_df.sum(axis=1)
wf_TC_China.to_csv("wf_results_2020.10/wf_TC_china.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_TD_China.to_csv("wf_results_2020.10/wf_TD_china.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_TC_p5_China.to_csv("wf_results_2020.10/wf_TC_p5_china.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_TD_p5_China.to_csv("wf_results_2020.10/wf_TD_p5_china.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_TC_p95_China.to_csv("wf_results_2020.10/wf_TC_p95_china.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_TD_p95_China.to_csv("wf_results_2020.10/wf_TD_p95_china.txt", sep="\t", header=True, index=True, float_format="%.2f")

# Calculate blue, green, gray WF for rice-produciton provinces

In [12]:
wf_mean_style = {}
wf_mean_style["TC"] = wf_mean_TC
wf_mean_style["TD"] = wf_mean_TD
for key in ["TC", "TD"]:
    wf_mean_df = wf_mean_style[key]
    wf_mean_df["Jiangxi"] = (wf_mean_df["Jiangxi2S"] * weight_yield_dic["Jiangxi2S"] + wf_mean_df["Jiangxi1S"] * weight_yield_dic["Jiangxi1S"]) / (
        weight_yield_dic["Jiangxi2S"] + weight_yield_dic["Jiangxi1S"])
    wf_mean_df["Fujian"] = (wf_mean_df["Fujian2S"] * weight_yield_dic["Fujian2S"] + wf_mean_df["Fujian1S"] * weight_yield_dic["Fujian1S"]) / (
        weight_yield_dic["Fujian2S"] + weight_yield_dic["Fujian1S"])
    wf_mean_df["Hunan"] = (wf_mean_df["Hunan2S"] * weight_yield_dic["Hunan2S"] + wf_mean_df["Hunan1S"] * weight_yield_dic["Hunan1S"]) / (
        weight_yield_dic["Hunan2S"] + weight_yield_dic["Hunan1S"])
    wf_mean_df.to_csv("wf_results_2020.10/wf_mean_provinces_{}.txt".format(key), sep="\t", header=True, index=True, float_format="%.2f")

# Calculate yield-weighted WF for three rice regions

In [13]:
weight_yield_df = pd.Series(weight_yield_dic)
weight_yield_df = weight_yield_df / sum_yield
weight_yield_df

Heilongjiang    0.137269
Jilin           0.032394
Liaoning        0.023116
Jiangsu         0.100161
Anhui           0.067206
Hubei           0.076623
Sichuan         0.104602
Guizhou         0.021274
Yunnan          0.027701
Jiangxi2S       0.086243
Hunan2S         0.094673
Hunan1S         0.047432
Jiangxi1S       0.023575
Zhejiang        0.020121
Fujian1S        0.009059
Guangdong       0.055894
Guangxi         0.052008
Hainan          0.007202
Fujian2S        0.013448
dtype: float64

In [14]:
region_dic = {"NE": ["Heilongjiang", "Jilin", "Liaoning"], "CJ": ["Jiangsu", "Anhui", "Hubei", "Sichuan", "Guizhou", "Yunnan", "Jiangxi2S", "Jiangxi1S",
                                                                 "Hunan2S", "Hunan1S"],
             "SE": ["Zhejiang", "Fujian1S", "Fujian2S", "Guangdong", "Guangxi", "Hainan"]}
# TC
wf_df = pd.read_csv("wf_results_2020.10/wf_mean_provinces_TC.txt", sep="\t", header=0, index_col=0)
wf_region_dic = {}
for region, provinces in region_dic.items():
    wf_df_provinces = wf_df[provinces]
    weight_series = weight_yield_df.loc[provinces]
    weight_sum = weight_series.sum()
    wf_weighted_df = wf_df_provinces * weight_series
    wf_weighted_sr = wf_weighted_df.sum(axis=1) / weight_sum
    wf_region_dic[region] = wf_weighted_sr
wf_weighted_df = pd.DataFrame(wf_region_dic)
wf_weighted_df.to_csv("wf_results_2020.10/wf_regions_TC.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_weighted_df

,NE,CJ,SE
bwf,441.319696,291.606167,279.919732
graywf,70.221704,437.327188,634.104236
greywf,0.000000,0.000000,0.000000
gwf,366.262812,408.799962,524.436549
leaching,10.342683,5.661990,6.004749
runoff,59.879021,431.665586,628.098913
runoff_N,59.593478,431.634935,627.328739
runoff_P,6.096275,62.360237,51.334192
runoff_f,138.607216,679.998491,764.222089
total,877.803014,1137.729135,1438.456645


In [15]:
# TD
wf_df = pd.read_csv("wf_results_2020.10/wf_mean_provinces_TD.txt", sep="\t", header=0, index_col=0)
wf_region_dic = {}
for region, provinces in region_dic.items():
    wf_df_provinces = wf_df[provinces]
    weight_series = weight_yield_df.loc[provinces]
    weight_sum = weight_series.sum()
    wf_weighted_df = wf_df_provinces * weight_series
    wf_weighted_sr = wf_weighted_df.sum(axis=1) / weight_sum
    wf_region_dic[region] = wf_weighted_sr
wf_weighted_df = pd.DataFrame(wf_region_dic)
wf_weighted_df.to_csv("wf_results_2020.10/wf_regions_TD.txt", sep="\t", header=True, index=True, float_format="%.2f")
wf_weighted_df

,NE,CJ,SE
bwf,395.112018,205.089408,192.693397
graywf,55.359742,257.434696,393.604181
greywf,-58.886725,-108.290427,-103.388621
gwf,351.133601,384.310752,506.583986
leaching,10.314659,5.539695,5.794435
runoff,45.046764,251.894430,387.809746
runoff_N,44.398932,251.833904,387.543283
runoff_P,4.993074,37.579879,29.410777
runoff_f,283.866990,877.339808,887.622127
total,801.598241,846.835101,1092.882595
